<a href="https://colab.research.google.com/github/DrUnicornIT/TSP-problem/blob/main/TSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TSProblem 


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
plt.style.use("seaborn")

## Take seed random py

In [2]:
np.random.seed(42)

## Make test01: easy

In [3]:
cities = [0, 1, 2, 3, 4]

adjacency_mat = np.asarray(
    [
        [0.00, 28.02, 17.12, 27.46, 46.07],
        [28.02, 0.00, 34.00, 25.55, 25.55],
        [17.12, 34.00, 0.00, 18.03, 57.38],
        [27.46, 25.55, 18.03, 0.00, 51.11],
        [46.07, 25.55, 57.38, 51.11, 0.00],
    ]
)

## Module TS

In [4]:
def swap(chromosome):
    a, b = np.random.choice(len(chromosome), 2)
    chromosome[a], chromosome[b] = (
        chromosome[b],
        chromosome[a]
    )
    return chromosome

class Population():
    def __init__(self, bag, adjacency_mat): 
        self.bag = bag  # all elements of the bag
        self.parents = []
        self.score = 0.00
        self.best = None
        self.adjacency_mat = adjacency_mat
    def fitness(self, chromosome): # fitness function cal ans for each chromosome
        return sum(
            [
                self.adjacency_mat[chromosome[i], chromosome[i + 1]]
                for i in range(len(chromosome) - 1)
            ]
        )
    def evaluate(self):
        distances = np.asarray(
            [self.fitness(chromosome) for chromosome in self.bag]
        )
        self.score = np.min(distances)
        self.best = self.bag[distances.tolist().index(self.score)]
        self.parents.append(self.best)
        if False in (distances[0] == distances):
            distances = np.max(distances) - distances
        return distances / np.sum(distances)
    def select(self, k=4):
        fit = self.evaluate()
        while len(self.parents) < k:
            idx = np.random.randint(0, len(fit))
            if fit[idx] > np.random.rand():
                self.parents.append(self.bag[idx])
        self.parents = np.asarray(self.parents)

    def crossover(self, p_cross=0.1):
        children = []
        count, size = self.parents.shape
        for i in range(len(self.bag)):
            if np.random.rand() < p_cross:
                children.append(
                    list(self.parents[np.random.randint(count, size=1)[0]])
                )
            else:
                parent1, parent2 = self.parents[
                    np.random.randint(count, size=2), :
                ]
                idx = np.random.choice(range(size), size=2, replace=False)
                start, end = min(idx), max(idx)
                child = [None] * size
                for i in range(start, end + 1, 1):
                    child[i] = parent1[i]
                pointer = 0
                for i in range(size):
                    if child[i] is None:
                        while parent2[pointer] in child:
                            pointer += 1
                        child[i] = parent2[pointer]
                children.append(child)
        return children
    def mutate(self, p_cross=0.1, p_mut=0.1):
        next_bag = []
        children = self.crossover(p_cross)
        for child in children:
            if np.random.rand() < p_mut:
                next_bag.append(swap(child))
            else:
                next_bag.append(child)
        return next_bag




## FIT: Module genetic_algorithm


In [5]:
def init_population(cities, adjacency_mat, n_population): 
        return Population(
            np.asarray([np.random.permutation(cities) for _ in range(n_population)]), adjacency_mat
            )

def genetic_algorithm(
    cities,
    adjacency_mat,
    n_population,
    n_iter=20,
    selectivity=0.15,
    p_cross=0.5,
    p_mut=0.1,
    print_interval=100,
    return_history=False,
    verbose=False,
):
    pop = init_population(cities, adjacency_mat, n_population)
    best = pop.best
    score = float("inf")
    history = []
    for i in tqdm(range(n_iter)):
        pop.select(n_population * selectivity)
        history.append(pop.score)
        if verbose:
            print(f"Generation {i}: {pop.score}")
        elif i % print_interval == 0:
            print(pop.best)
            print(f"Generation {i}: {pop.score}")
        if pop.score < score:
            best = pop.best
            score = pop.score
        children = pop.mutate(p_cross, p_mut)
        pop = Population(children, pop.adjacency_mat)
    if return_history:
        return best, history
    return Best

## Test-basic

In [6]:
def Test_Basic01() : 
    cities = [0, 1, 2, 3, 4]
    adjacency_mat = np.asarray(
        [
            [0.00, 28.02, 17.12, 27.46, 46.07],
            [28.02, 0.00, 34.00, 25.55, 25.55],
            [17.12, 34.00, 0.00, 18.03, 57.38],
            [27.46, 25.55, 18.03, 0.00, 51.11],
            [46.07, 25.55, 57.38, 51.11, 0.00],
        ]
    )
    pop = init_population(cities, adjacency_mat, 6)

    print("bag")
    print(pop.bag)
    print("evaluate")
    pop.evaluate()
    print("Best")
    print(pop.best)
    
    pop.select()
    print("parents")
    print(pop.parents)
    print("mutate")
    print(pop.mutate())

In [7]:
Test_Basic01()

bag
[[1 4 2 0 3]
 [3 1 2 0 4]
 [1 0 3 4 2]
 [0 1 2 3 4]
 [0 2 3 1 4]
 [4 2 0 3 1]]
evaluate
Best
[0 2 3 1 4]
parents
[[0 2 3 1 4]
 [0 2 3 1 4]
 [3 1 2 0 4]
 [4 2 0 3 1]]
mutate
[[3, 2, 0, 1, 4], [2, 3, 1, 0, 4], [0, 2, 3, 1, 4], [4, 2, 0, 3, 1], [0, 2, 3, 1, 4], [0, 2, 3, 4, 1]]


## Test-advance

In [8]:
def Test_Adv01 ():
    def make_mat(coordinates):
        res = [
            [get_distance(city1, city2) for city2 in coordinates]
        for city1 in coordinates
        ]
        return np.asarray(res)

    def get_distance(city1, city2):
        return np.sqrt((city1[0] - city2[0])**2 + (city1[1] - city2[1])**2)

    def better_generate_cities(n_cities, factor=0.2):
        x = np.asarray(range(int(-n_cities / 2), int(n_cities / 2) + 1, 1))
        y = np.sqrt(n_cities ** 2 / 4 - x ** 2)
        return np.asarray(list(zip(x, y)))

    def print_path(best, city_coordinates):
        points = city_coordinates[best]
        x, y = zip(*points)
        plt.plot(x, y, color="skyblue", marker="o")
        plt.show()

    cities = range(100)
    city_coordinates = better_generate_cities(len(cities))
    adjacency_mat = make_mat(city_coordinates)
    best, history = genetic_algorithm(
        cities, adjacency_mat, n_population=500, selectivity=0.05,
        p_mut=0.05, p_cross=0.7, n_iter=8000, print_interval=100, verbose=False, return_history=True
    )

    plt.plot(range(len(history)), history, color="skyblue")
    plt.show()
    print(best)
    print(history)
    print_path(best, city_coordinates)
    print_path(sorted(best), city_coordinates)


In [ ]:
Test_Adv01()


  0%|          | 1/8000 [00:00<35:18,  3.78it/s]

[94 87 56 63 35 24 50 76 57 93 45 82 97 95 77 79 80 58 20 52 37 85 83  3
 17 44 67 73 74 54 66 41 16 64  5 31 36 25 89 96 61 99  7  9 60 70 22 21
 43 28 81 34 14 46 47 72 88 65 91 39 62 49 51 84 78 48 29 18 33 68 23 12
 13  1 90 32 75 26 42 38  8  6 86 30 15 98 27  2 10 53 92 69 55  0 71 19
 40 59  4 11]
Generation 0: 3136.075685414582


  1%|▏         | 101/8000 [00:27<17:30,  7.52it/s]

[94, 87, 80, 58, 32, 30, 99, 98, 97, 82, 85, 89, 93, 91, 90, 79, 71, 0, 15, 20, 17, 67, 73, 74, 54, 41, 16, 5, 35, 36, 61, 60, 13, 6, 7, 12, 8, 34, 42, 65, 55, 66, 56, 43, 39, 46, 63, 78, 96, 95, 84, 68, 50, 45, 37, 31, 25, 22, 21, 19, 23, 10, 9, 1, 2, 3, 27, 51, 59, 44, 52, 81, 70, 75, 76, 77, 86, 92, 64, 53, 57, 11, 4, 18, 14, 28, 72, 88, 62, 49, 47, 48, 29, 33, 24, 26, 38, 40, 69, 83]
Generation 100: 1507.6585185637462


  3%|▎         | 202/8000 [00:40<17:08,  7.59it/s]

...

  4%|▍         | 302/8000 [00:53<16:30,  7.77it/s]

[80, 72, 51, 38, 87, 94, 97, 98, 99, 85, 89, 93, 91, 90, 82, 79, 8, 9, 15, 17, 20, 73, 74, 67, 65, 61, 54, 41, 35, 36, 60, 58, 30, 16, 4, 6, 7, 12, 34, 43, 40, 39, 46, 78, 96, 95, 92, 84, 81, 68, 50, 45, 42, 37, 31, 25, 27, 22, 21, 19, 14, 10, 0, 1, 2, 3, 5, 32, 52, 55, 56, 62, 63, 66, 70, 75, 76, 77, 86, 88, 83, 64, 53, 57, 11, 13, 18, 23, 24, 29, 71, 69, 59, 49, 47, 48, 44, 33, 28, 26]
Generation 300: 1054.692308327317


  5%|▌         | 401/8000 [01:06<17:06,  7.41it/s]

[80, 79, 87, 94, 97, 98, 99, 89, 93, 91, 90, 85, 8, 9, 12, 15, 17, 20, 73, 74, 67, 65, 61, 60, 58, 54, 52, 41, 35, 36, 30, 29, 16, 4, 6, 7, 34, 38, 40, 39, 46, 59, 78, 96, 95, 92, 84, 82, 81, 72, 68, 50, 45, 42, 37, 31, 25, 22, 21, 19, 14, 10, 0, 1, 2, 3, 5, 32, 43, 55, 56, 62, 63, 66, 70, 75, 76, 77, 86, 88, 83, 71, 69, 64, 53, 49, 11, 13, 18, 23, 24, 27, 51, 57, 47, 48, 44, 33, 28, 26]
Generation 400: 863.4713600209682


  5%|▌         | 422/8000 [01:09<16:13,  7.78it/s]